In [1]:
import pandas as pd

In [327]:
# мой файл
df_train = pd.read_pickle('df_train.p')

In [328]:
df_train['count'] = df_train['count'].fillna(0)
df_train['type'] = df_train['type'].fillna('no')

In [329]:
df_train = df_train.drop(['order_created_at', 'shipment_starts_at', 'shipped_at','user_id','ship_address_id','shipment_id','order_id','shipment_state','dw_kind'], axis=1)

In [330]:
df_TRAIN = df_train

In [331]:
df_TRAIN = df_TRAIN.fillna(0)

In [332]:
df_TRAIN = df_TRAIN.drop(['s.city_name', 'target'], axis=1)

In [333]:
df_TRAIN = pd.get_dummies(data=df_TRAIN, columns=['retailer', 's.order_state', 'platform', 'os', 'type'])

In [334]:
# для прогноза
df_august = df_TRAIN[df_TRAIN['order_completed_at'] == '2020-08'].copy()

In [336]:
# для обучения
df_TRAIN = df_TRAIN[df_TRAIN['order_completed_at'] != '2020-08'].copy()

In [337]:
df_target = pd.read_csv('ngwl-predict-customer-churn/train/train.csv')

In [338]:
df_target.order_completed_at = pd.to_datetime(df_target.order_completed_at).dt.to_period('M')

In [339]:
df_TRAIN = df_TRAIN.merge(df_target, on=['phone_id', 'order_completed_at'])

In [340]:
df_TRAIN = df_TRAIN.drop(['phone_id','order_completed_at'], axis=1)

In [341]:
df_TRAIN

,s.store_id,total_cost,rate,promo_total,total_weight,parent_category_id,count,retailer_0,retailer_BILLA,retailer_Home Market,...,os_ios,os_linux,os_mac,os_other,os_windows,type_email,type_no,type_push,type_sms,target
0,87.0,98.0,0.0,0.0,9710.0,20.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,0.0
1,87.0,0.0,0.0,-2000.0,9306.0,9.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,0.0
2,100.0,98.0,0.0,0.0,6150.0,20.0,0.0,0,0,0,...,0,0,1,0,0,0,1,0,0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0.0
4,257.0,98.0,0.0,0.0,977.0,56.0,1.0,0,0,0,...,0,0,1,0,0,1,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731079,12.0,158.0,0.0,0.0,1280.0,119.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,1.0
1731080,3.0,158.0,0.0,0.0,1280.0,119.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,1.0
1731081,12.0,158.0,0.0,0.0,2000.0,119.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,1.0
1731082,205.0,99.0,0.0,0.0,1330.0,112.0,0.0,0,0,0,...,0,0,0,0,1,0,1,0,0,1.0


In [342]:
import numpy as np
from sklearn.model_selection import train_test_split

In [343]:
X = df_TRAIN.iloc[:,:-1]
y = df_TRAIN.iloc[:,-1]

In [397]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.9)

In [398]:
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [401]:
model2 = CatBoostClassifier(iterations=1000, learning_rate=0.01, l2_leaf_reg=3.5, depth=8, rsm=0.98, loss_function= 'CrossEntropy', eval_metric='AUC',use_best_model=True)

In [402]:
model2.fit(xtrain,ytrain,eval_set=(xtest,ytest))

0:	test: 0.7379353	best: 0.7379353 (0)	total: 283ms	remaining: 9m 25s
1:	test: 0.7385339	best: 0.7385339 (1)	total: 550ms	remaining: 9m 8s
2:	test: 0.7390341	best: 0.7390341 (2)	total: 825ms	remaining: 9m 8s
3:	test: 0.7389491	best: 0.7390341 (2)	total: 1.03s	remaining: 8m 35s
4:	test: 0.7410775	best: 0.7410775 (4)	total: 1.25s	remaining: 8m 20s
5:	test: 0.7414779	best: 0.7414779 (5)	total: 1.52s	remaining: 8m 25s
6:	test: 0.7410983	best: 0.7414779 (5)	total: 1.79s	remaining: 8m 30s
7:	test: 0.7411444	best: 0.7414779 (5)	total: 2.04s	remaining: 8m 27s
8:	test: 0.7417170	best: 0.7417170 (8)	total: 2.31s	remaining: 8m 30s
9:	test: 0.7419731	best: 0.7419731 (9)	total: 2.57s	remaining: 8m 31s
10:	test: 0.7424166	best: 0.7424166 (10)	total: 2.83s	remaining: 8m 32s
11:	test: 0.7427611	best: 0.7427611 (11)	total: 3.09s	remaining: 8m 31s
12:	test: 0.7428467	best: 0.7428467 (12)	total: 3.35s	remaining: 8m 32s
13:	test: 0.7429052	best: 0.7429052 (13)	total: 3.61s	remaining: 8m 32s
14:	test: 0.74

114:	test: 0.7455237	best: 0.7455237 (114)	total: 32s	remaining: 8m 44s
115:	test: 0.7456047	best: 0.7456047 (115)	total: 32.3s	remaining: 8m 43s
116:	test: 0.7456124	best: 0.7456124 (116)	total: 32.5s	remaining: 8m 43s
117:	test: 0.7456500	best: 0.7456500 (117)	total: 32.8s	remaining: 8m 43s
118:	test: 0.7457649	best: 0.7457649 (118)	total: 33.1s	remaining: 8m 43s
119:	test: 0.7457406	best: 0.7457649 (118)	total: 33.4s	remaining: 8m 43s
120:	test: 0.7457264	best: 0.7457649 (118)	total: 33.7s	remaining: 8m 43s
121:	test: 0.7458347	best: 0.7458347 (121)	total: 34s	remaining: 8m 43s
122:	test: 0.7459418	best: 0.7459418 (122)	total: 34.3s	remaining: 8m 43s
123:	test: 0.7459384	best: 0.7459418 (122)	total: 34.6s	remaining: 8m 43s
124:	test: 0.7459700	best: 0.7459700 (124)	total: 34.8s	remaining: 8m 42s
125:	test: 0.7459876	best: 0.7459876 (125)	total: 35.1s	remaining: 8m 42s
126:	test: 0.7460239	best: 0.7460239 (126)	total: 35.4s	remaining: 8m 41s
127:	test: 0.7460166	best: 0.7460239 (126)

226:	test: 0.7486569	best: 0.7486569 (226)	total: 1m 3s	remaining: 8m 17s
227:	test: 0.7486712	best: 0.7486712 (227)	total: 1m 3s	remaining: 8m 17s
228:	test: 0.7486988	best: 0.7486988 (228)	total: 1m 4s	remaining: 8m 16s
229:	test: 0.7487175	best: 0.7487175 (229)	total: 1m 4s	remaining: 8m 16s
230:	test: 0.7487360	best: 0.7487360 (230)	total: 1m 4s	remaining: 8m 16s
231:	test: 0.7487414	best: 0.7487414 (231)	total: 1m 5s	remaining: 8m 16s
232:	test: 0.7487687	best: 0.7487687 (232)	total: 1m 5s	remaining: 8m 15s
233:	test: 0.7487886	best: 0.7487886 (233)	total: 1m 5s	remaining: 8m 15s
234:	test: 0.7488027	best: 0.7488027 (234)	total: 1m 6s	remaining: 8m 15s
235:	test: 0.7488150	best: 0.7488150 (235)	total: 1m 6s	remaining: 8m 15s
236:	test: 0.7488146	best: 0.7488150 (235)	total: 1m 6s	remaining: 8m 15s
237:	test: 0.7488467	best: 0.7488467 (237)	total: 1m 6s	remaining: 8m 15s
238:	test: 0.7488498	best: 0.7488498 (238)	total: 1m 7s	remaining: 8m 15s
239:	test: 0.7489006	best: 0.7489006 (

337:	test: 0.7507169	best: 0.7507169 (337)	total: 1m 35s	remaining: 7m 48s
338:	test: 0.7507254	best: 0.7507254 (338)	total: 1m 35s	remaining: 7m 48s
339:	test: 0.7507590	best: 0.7507590 (339)	total: 1m 35s	remaining: 7m 48s
340:	test: 0.7507734	best: 0.7507734 (340)	total: 1m 36s	remaining: 7m 47s
341:	test: 0.7507952	best: 0.7507952 (341)	total: 1m 36s	remaining: 7m 47s
342:	test: 0.7508008	best: 0.7508008 (342)	total: 1m 36s	remaining: 7m 47s
343:	test: 0.7508136	best: 0.7508136 (343)	total: 1m 37s	remaining: 7m 47s
344:	test: 0.7508157	best: 0.7508157 (344)	total: 1m 37s	remaining: 7m 46s
345:	test: 0.7508234	best: 0.7508234 (345)	total: 1m 37s	remaining: 7m 46s
346:	test: 0.7508474	best: 0.7508474 (346)	total: 1m 37s	remaining: 7m 46s
347:	test: 0.7508570	best: 0.7508570 (347)	total: 1m 38s	remaining: 7m 45s
348:	test: 0.7508602	best: 0.7508602 (348)	total: 1m 38s	remaining: 7m 45s
349:	test: 0.7508729	best: 0.7508729 (349)	total: 1m 38s	remaining: 7m 45s
350:	test: 0.7508741	best

447:	test: 0.7520654	best: 0.7520654 (447)	total: 2m 6s	remaining: 7m 17s
448:	test: 0.7520851	best: 0.7520851 (448)	total: 2m 6s	remaining: 7m 17s
449:	test: 0.7520937	best: 0.7520937 (449)	total: 2m 6s	remaining: 7m 16s
450:	test: 0.7521091	best: 0.7521091 (450)	total: 2m 7s	remaining: 7m 16s
451:	test: 0.7521135	best: 0.7521135 (451)	total: 2m 7s	remaining: 7m 16s
452:	test: 0.7521182	best: 0.7521182 (452)	total: 2m 7s	remaining: 7m 16s
453:	test: 0.7521231	best: 0.7521231 (453)	total: 2m 7s	remaining: 7m 15s
454:	test: 0.7521370	best: 0.7521370 (454)	total: 2m 8s	remaining: 7m 15s
455:	test: 0.7521469	best: 0.7521469 (455)	total: 2m 8s	remaining: 7m 15s
456:	test: 0.7521653	best: 0.7521653 (456)	total: 2m 8s	remaining: 7m 15s
457:	test: 0.7521699	best: 0.7521699 (457)	total: 2m 9s	remaining: 7m 14s
458:	test: 0.7521956	best: 0.7521956 (458)	total: 2m 9s	remaining: 7m 14s
459:	test: 0.7521997	best: 0.7521997 (459)	total: 2m 9s	remaining: 7m 14s
460:	test: 0.7522154	best: 0.7522154 (

...

1536:	test: 0.7568792	best: 0.7568792 (1536)	total: 7m 28s	remaining: 2m 15s
1537:	test: 0.7568831	best: 0.7568831 (1537)	total: 7m 28s	remaining: 2m 14s
1538:	test: 0.7568863	best: 0.7568863 (1538)	total: 7m 29s	remaining: 2m 14s
1539:	test: 0.7568902	best: 0.7568902 (1539)	total: 7m 29s	remaining: 2m 14s
1540:	test: 0.7568954	best: 0.7568954 (1540)	total: 7m 29s	remaining: 2m 13s
1541:	test: 0.7568974	best: 0.7568974 (1541)	total: 7m 30s	remaining: 2m 13s
1542:	test: 0.7568986	best: 0.7568986 (1542)	total: 7m 30s	remaining: 2m 13s
1543:	test: 0.7568983	best: 0.7568986 (1542)	total: 7m 30s	remaining: 2m 13s
1544:	test: 0.7569013	best: 0.7569013 (1544)	total: 7m 30s	remaining: 2m 12s
1545:	test: 0.7569042	best: 0.7569042 (1545)	total: 7m 31s	remaining: 2m 12s
1546:	test: 0.7569066	best: 0.7569066 (1546)	total: 7m 31s	remaining: 2m 12s
1547:	test: 0.7569093	best: 0.7569093 (1547)	total: 7m 31s	remaining: 2m 11s
1548:	test: 0.7569124	best: 0.7569124 (1548)	total: 7m 32s	remaining: 2m 11s

1644:	test: 0.7571483	best: 0.7571483 (1644)	total: 8m 5s	remaining: 1m 44s
1645:	test: 0.7571494	best: 0.7571494 (1645)	total: 8m 5s	remaining: 1m 44s
1646:	test: 0.7571513	best: 0.7571513 (1646)	total: 8m 5s	remaining: 1m 44s
1647:	test: 0.7571561	best: 0.7571561 (1647)	total: 8m 6s	remaining: 1m 43s
1648:	test: 0.7571568	best: 0.7571568 (1648)	total: 8m 6s	remaining: 1m 43s
1649:	test: 0.7571598	best: 0.7571598 (1649)	total: 8m 6s	remaining: 1m 43s
1650:	test: 0.7571626	best: 0.7571626 (1650)	total: 8m 7s	remaining: 1m 43s
1651:	test: 0.7571651	best: 0.7571651 (1651)	total: 8m 7s	remaining: 1m 42s
1652:	test: 0.7571674	best: 0.7571674 (1652)	total: 8m 8s	remaining: 1m 42s
1653:	test: 0.7571691	best: 0.7571691 (1653)	total: 8m 8s	remaining: 1m 42s
1654:	test: 0.7571695	best: 0.7571695 (1654)	total: 8m 8s	remaining: 1m 41s
1655:	test: 0.7571747	best: 0.7571747 (1655)	total: 8m 9s	remaining: 1m 41s
1656:	test: 0.7571763	best: 0.7571763 (1656)	total: 8m 9s	remaining: 1m 41s
1657:	test: 

1751:	test: 0.7573874	best: 0.7573874 (1751)	total: 8m 37s	remaining: 1m 13s
1752:	test: 0.7573904	best: 0.7573904 (1752)	total: 8m 38s	remaining: 1m 13s
1753:	test: 0.7573943	best: 0.7573943 (1753)	total: 8m 38s	remaining: 1m 12s
1754:	test: 0.7573993	best: 0.7573993 (1754)	total: 8m 38s	remaining: 1m 12s
1755:	test: 0.7574018	best: 0.7574018 (1755)	total: 8m 39s	remaining: 1m 12s
1756:	test: 0.7574032	best: 0.7574032 (1756)	total: 8m 39s	remaining: 1m 11s
1757:	test: 0.7574052	best: 0.7574052 (1757)	total: 8m 40s	remaining: 1m 11s
1758:	test: 0.7574076	best: 0.7574076 (1758)	total: 8m 40s	remaining: 1m 11s
1759:	test: 0.7574085	best: 0.7574085 (1759)	total: 8m 40s	remaining: 1m 11s
1760:	test: 0.7574092	best: 0.7574092 (1760)	total: 8m 41s	remaining: 1m 10s
1761:	test: 0.7574120	best: 0.7574120 (1761)	total: 8m 41s	remaining: 1m 10s
1762:	test: 0.7574151	best: 0.7574151 (1762)	total: 8m 41s	remaining: 1m 10s
1763:	test: 0.7574172	best: 0.7574172 (1763)	total: 8m 42s	remaining: 1m 9s


1860:	test: 0.7576151	best: 0.7576151 (1860)	total: 9m 14s	remaining: 41.4s
1861:	test: 0.7576160	best: 0.7576160 (1861)	total: 9m 14s	remaining: 41.1s
1862:	test: 0.7576168	best: 0.7576168 (1862)	total: 9m 15s	remaining: 40.8s
1863:	test: 0.7576197	best: 0.7576197 (1863)	total: 9m 15s	remaining: 40.5s
1864:	test: 0.7576219	best: 0.7576219 (1864)	total: 9m 15s	remaining: 40.2s
1865:	test: 0.7576252	best: 0.7576252 (1865)	total: 9m 16s	remaining: 39.9s
1866:	test: 0.7576287	best: 0.7576287 (1866)	total: 9m 16s	remaining: 39.6s
1867:	test: 0.7576300	best: 0.7576300 (1867)	total: 9m 16s	remaining: 39.3s
1868:	test: 0.7576314	best: 0.7576314 (1868)	total: 9m 17s	remaining: 39s
1869:	test: 0.7576323	best: 0.7576323 (1869)	total: 9m 17s	remaining: 38.7s
1870:	test: 0.7576330	best: 0.7576330 (1870)	total: 9m 17s	remaining: 38.4s
1871:	test: 0.7576349	best: 0.7576349 (1871)	total: 9m 17s	remaining: 38.2s
1872:	test: 0.7576373	best: 0.7576373 (1872)	total: 9m 18s	remaining: 37.9s
1873:	test: 0.

1969:	test: 0.7578068	best: 0.7578068 (1969)	total: 9m 48s	remaining: 8.96s
1970:	test: 0.7578085	best: 0.7578085 (1970)	total: 9m 48s	remaining: 8.66s
1971:	test: 0.7578130	best: 0.7578130 (1971)	total: 9m 48s	remaining: 8.36s
1972:	test: 0.7578148	best: 0.7578148 (1972)	total: 9m 49s	remaining: 8.06s
1973:	test: 0.7578158	best: 0.7578158 (1973)	total: 9m 49s	remaining: 7.76s
1974:	test: 0.7578168	best: 0.7578168 (1974)	total: 9m 49s	remaining: 7.46s
1975:	test: 0.7578186	best: 0.7578186 (1975)	total: 9m 50s	remaining: 7.17s
1976:	test: 0.7578205	best: 0.7578205 (1976)	total: 9m 50s	remaining: 6.87s
1977:	test: 0.7578227	best: 0.7578227 (1977)	total: 9m 50s	remaining: 6.57s
1978:	test: 0.7578234	best: 0.7578234 (1978)	total: 9m 50s	remaining: 6.27s
1979:	test: 0.7578262	best: 0.7578262 (1979)	total: 9m 51s	remaining: 5.97s
1980:	test: 0.7578278	best: 0.7578278 (1980)	total: 9m 51s	remaining: 5.67s
1981:	test: 0.7578291	best: 0.7578291 (1981)	total: 9m 51s	remaining: 5.37s
1982:	test: 

In [369]:
# таблица с id, которым нужно предсказать
df_data = pd.read_csv('ngwl-predict-customer-churn/sample_submission.csv', sep=';')

In [371]:
df_data = df_data.rename({'Id': 'phone_id'}, axis=1).merge(df_august, how='left', on=['phone_id'])

In [372]:
df_data = df_data.drop(['Predicted', 'order_completed_at'], axis=1)

In [373]:
df_data = df_data.dropna()

In [374]:
# вспомогательная таблица (нужна бедет дальше)
df_id = df_data[['phone_id']]

In [375]:
df_data = df_data.drop(['phone_id'], axis=1)

In [376]:
df_data

,s.store_id,total_cost,rate,promo_total,total_weight,parent_category_id,count,retailer_0,retailer_BILLA,retailer_Home Market,...,os_android,os_ios,os_linux,os_mac,os_other,os_windows,type_email,type_no,type_push,type_sms
0,110.0,98.0,5.0,-300.0,12554.0,74.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,65.0,49.0,0.0,-250.0,6650.0,20.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,211.0,158.0,5.0,0.0,12300.0,9.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,406.0,80.0,5.0,0.0,49500.0,20.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392125,81.0,158.0,0.0,0.0,9100.0,508.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
392126,12.0,158.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
392127,12.0,158.0,0.0,0.0,5500.0,126.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
392128,12.0,158.0,0.0,0.0,1600.0,119.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [403]:
preds_class = model2.predict(df_data)

In [405]:
df_id['Predicted'] = preds_class

In [406]:
df_id

,phone_id,Predicted
0,19843,1
1,19843,1
3,342522,0
4,457799,1
5,457799,1
...,...,...
392125,23853,1
392126,23853,1
392127,23853,1
392128,23853,1


In [ ]:
# Дальше происходит дичь с дубликатами... понимать не нужно

In [407]:
df_d = pd.read_csv('ngwl-predict-customer-churn/sample_submission.csv', sep=';')

In [408]:
df_d = df_d.drop(['Predicted'], axis=1)

In [409]:
df_d = df_d.merge(df_id.rename({'phone_id': 'Id'}, axis=1), how='left', on=['Id'])

In [410]:
df_d = df_d.drop_duplicates()

In [411]:
df_d = df_d.dropna()

In [412]:
df_d = df_d.sort_values('Predicted')

In [413]:
df_d = df_d.drop_duplicates('Id', keep='last')

In [414]:
df = pd.read_csv('ngwl-predict-customer-churn/sample_submission.csv', sep=';')

In [416]:
df = df.drop(['Predicted'], axis=1).merge(df_d, how='left', on=['Id'])

In [ ]:
df = df.fillna(1)

In [326]:
df.to_csv('predicted_new1.csv')